## Importing Data from Wikipedia

Import relevant packages

In [3]:
import numpy as np
import pandas as pd
import json
import requests

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [ ]:
!conda install -c conda-forge geopy --yes

In [18]:
from geopy.geocoders import Nominatim

Assign Wikipedia Link for Toronto Postal Codes to "url"

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Read HTML from URL and assign it to the toronto_data variable

In [5]:
data = pd.read_html(url)
toronto_data = data[0]

Drop rows in the table where the borough is not assigned

In [6]:
toronto_data = toronto_data[toronto_data["Borough"]!="Not assigned"]

Reset the index of the table

In [7]:
toronto_data.reset_index(drop=True,inplace=True)

Allow all table rows and columns to be displayed

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

Check number of rows and columns in the toronto_data dataframe

In [9]:
toronto_data.shape

(103, 3)

In [10]:
toronto_data

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Importing and Adding Latitude and Longitude Data

In [11]:
!wget -q -O "geospatial_data.csv" https://cocl.us/Geospatial_data

In [12]:
geo_data = pd.read_csv("geospatial_data.csv")

In [13]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
toronto_loc_data = pd.merge(toronto_data, geo_data, on='Postal Code', how='outer')

In [15]:
toronto_loc_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
toronto_loc_data.shape

(103, 5)

Check the number of boroughs and neighborhoods in our dataframe

In [24]:
print("The dataframe has {} boroughs".format(len(toronto_loc_data["Borough"].unique())))

The dataframe has 10 boroughs


Let's visualize the boroughs we currently have using Folium. First, let's get the coordinates of the city of Toronto.

In [25]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a Map of Toronto with the Postal Codes superimposed on top 

In [30]:
#create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location = [latitude,longitude], zoom_start=10)

#add markers to map
for lat, lng, pc, borough in zip(toronto_loc_data["Latitude"], toronto_loc_data["Longitude"],\
                                 toronto_loc_data["Postal Code"], toronto_loc_data["Borough"]):
    label = "{}, {}".format(pc, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto;

## Segmenting Data

As per the recommendation, let us select only the Boroughs that contain the name "Toronto"

In [47]:
to_mask = toronto_loc_data["Borough"].str.find("Toronto")>0
df_to = toronto_loc_data[to_mask]

In [48]:
df_to.reset_index(drop=True,inplace=True)

In [52]:
df_to.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Next, use Foursquare API to explore and segment the postal codes

### Define Foursquare Credentials and Version

In [53]:
CLIENT_ID = 'RMGLSZNSF0E2O4ZDM2RUE54HPXOYLX105EHCSIOAZYKGINFM' # your Foursquare ID
CLIENT_SECRET = 'NK1W0BWWHDOEO4Q1XG5KNCK2PUIDNEFKINVYV1SCU42FHIY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: RMGLSZNSF0E2O4ZDM2RUE54HPXOYLX105EHCSIOAZYKGINFM
CLIENT_SECRET:NK1W0BWWHDOEO4Q1XG5KNCK2PUIDNEFKINVYV1SCU42FHIY1


Let's look at the first postal code in the dataframe

In [54]:
df_to.loc[0, "Postal Code"]

'M5A'

Get the postal code's latitude and longitude values

In [56]:
pc_lat = df_to.loc[0, 'Latitude']
pc_lng = df_to.loc[0, 'Longitude']
pc_name = df_to.loc[0, "Postal Code"]

print("Latitude and Longitude values of postal code {} are {}, {}".format(pc_name, pc_lat, pc_lng))

Latitude and Longitude values of postal code M5A are 43.6542599, -79.3606359


### As in the lab, let's get the first 100 venues that are in postal code M5A within a 500 meter radius

Create GET request URL

In [57]:
LIMIT = 100
radius = 500
url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&\
client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    pc_lat,
    pc_lng,
    radius,
    LIMIT)

Send the GET request and examine the results

In [60]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eea6a46226d4f12e82bc61a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Define a function to extract the category from the JSON response

In [68]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
venues = results['response']['groups'][0]['items']

Flatten the JSON file into a pandas dataframe using pd.json_normalize, create a dataframe of the venues with their category and coordinates

In [81]:
nearby_venues = pd.json_normalize(venues) #Flatten JSON file

#Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

#Extract the category for each data entry
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)

#Clean the dataframe titles
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


In [84]:
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Corktown Common,Park,43.655618,-79.356211


### Let's repeat the process for all of the postal codes in the city of Toronto

This method is from the NYC Segmenting and Clustering Lab

In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
toronto_venues = getNearbyVenues(names = df_to["Postal Code"],
                                latitudes = df_to["Latitude"],
                                longitudes= df_to["Longitude"])

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [89]:
toronto_venues.head()

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [90]:
print(toronto_venues.shape)

(1627, 7)


Check number of venues returned for each postal code

In [91]:
toronto_venues.groupby("Postal Code").count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,4,4,4,4,4,4
M4K,43,43,43,43,43,43
M4L,23,23,23,23,23,23
M4M,40,40,40,40,40,40
M4N,3,3,3,3,3,3
M4P,8,8,8,8,8,8
M4R,19,19,19,19,19,19
M4S,35,35,35,35,35,35
M4T,1,1,1,1,1,1


How many unique categories are there?

In [95]:
print("There are {} unique categories of venues in Toronto.".format(len(toronto_venues["Venue Category"].unique())))

There are 240 unique categories of venues in Toronto.


## Analyze each postal code using one-hot encoding

In [99]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="", prefix_sep="")

#add postal code column back to onehot dataframe
toronto_onehot["Postal Code"]=toronto_venues["Postal Code"]

#move postal code column to first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [100]:
toronto_onehot.head()

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group rows by postal code and find the mean frequency occurrence of each category:

In [101]:
toronto_grouped = toronto_onehot.groupby("Postal Code").mean().reset_index()
toronto_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00

In [102]:
toronto_grouped.shape

(39, 241)

Let's print each postal code with the top 5 venues in each

In [118]:
num_top = 5

for p in toronto_grouped["Postal Code"]:
    print("----"+p+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code']== p].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top))
    print('\n')

----M4E----
               venue  freq
0       Neighborhood  0.25
1              Trail  0.25
2  Health Food Store  0.25
3                Pub  0.25
4      Movie Theater  0.00


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4      Frozen Yogurt Shop  0.05


----M4L----
                  venue  freq
0                  Park  0.13
1  Fast Food Restaurant  0.09
2             Pet Store  0.04
3           Coffee Shop  0.04
4         Movie Theater  0.04


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3  American Restaurant  0.05
4            Gastropub  0.05


----M4N----
               venue  freq
0               Park  0.33
1           Bus Line  0.33
2        Swim School  0.33
3  Afghan Restaurant  0.00
4             Museum  0.00


----M4P----
               venue  freq
0              Hotel 

### Put into a pandas dataframe

Function to sort venues in descending order

In [119]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create new dataframe listing top 10 venues in each postal code

In [165]:
num_top_venues = 10

suffixes = ['st','nd','rd']

# create columns according to rank of venue frequency
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,suffixes[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['Postal Code']=toronto_grouped["Postal Code"]

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)

pc_venues_sorted;

## Cluster Postal Codes

Use k-means to cluster the postal codes

In [166]:
#set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop("Postal Code",1)

#run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

#check cluster labels generated for each row in dataframe
len(kmeans.labels_)

39

Create a new dataframe that includes the cluster and the top 10 venues for each neighborhood

In [167]:
#add clustering labels
pc_venues_sorted.insert(0, "Cluster Label",kmeans.labels_)

toronto_merged = df_to

#merge toronto_grouped with toronto_loc_data to add lat/long for each postal code
toronto_merged = toronto_merged.join(pc_venues_sorted.set_index("Postal Code"), on="Postal Code")

In [149]:
toronto_merged;

Visualize resulting clusters

In [170]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [160]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Park,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
21,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,0,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


These venues seem to have most access to Parks and Playgrounds

#### Cluster 2

In [161]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Bank,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Japanese Restaurant,Tea Room,Bookstore
3,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gastropub,Restaurant,Italian Restaurant,Creperie,Lingerie Store,Moroccan Restaurant
4,East Toronto,1,Neighborhood,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Café,Beer Bar,Jazz Club,Hotel
6,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Salad Place,Dessert Shop,Middle Eastern Restaurant,Thai Restaurant,Department Store
7,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Candy Store,Baby Store,Diner,Italian Restaurant,Coffee Shop,Nightclub
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Thai Restaurant,Gym,Deli / Bodega,Hotel,Steakhouse,Pizza Place,Sushi Restaurant
9,West Toronto,1,Pharmacy,Bakery,Brewery,Middle Eastern Restaurant,Bar,Café,Furniture / Home Store,Bank,Music Venue,Park


This cluster seems to cater most to coffee shops, cafes and shopping.

#### Cluster 3

In [162]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Home Service,Music Venue,Garden,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


There seem to be many home service venues in this cluster area.

#### Cluster 4

In [163]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,3,Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


This looks like the only cluster near tennis courts.